In [1]:
from __future__ import print_function
from distutils.version import LooseVersion as Version
import sys

OK = '\x1b[42m[ OK ]\x1b[0m'
FAIL = "\x1b[41m[FAIL]\x1b[0m"

try:
    import importlib
except ImportError:
    print(FAIL, "Python version 3.9 is required,"
                " but %s is installed." % sys.version)

def import_version(pkg, min_ver, fail_msg=""):
    mod = None
    try:
        mod = importlib.import_module(pkg)
        if pkg in {'PIL'}:
            ver = mod.VERSION
        else:
            ver = mod.__version__
        if Version(ver) == min_ver:
            print(OK, "%s version %s is installed."
                  % (lib, min_ver))
        else:
            print(FAIL, "%s version %s is required, but %s installed."
                  % (lib, min_ver, ver))    
    except ImportError:
        print(FAIL, '%s not installed. %s' % (pkg, fail_msg))
    return mod


# first check the python version
pyversion = Version(sys.version)
if pyversion >= "3.9":
    print(OK, "Python version is %s" % sys.version)
elif pyversion < "3.9":
    print(FAIL, "Python version 3.9 is required,"
                " but %s is installed." % sys.version)
else:
    print(FAIL, "Unknown Python version: %s" % sys.version)

    
print()
requirements = {'numpy': "1.21.1", 'matplotlib': "3.4.2",'sklearn': "0.24.2", 
                'pandas': "1.3.1",'xgboost': "1.3.3", 'shap': "0.39.0"}

# now the dependencies
for lib, required_version in list(requirements.items()):
    import_version(lib, required_version)

[ OK ] Python version is 3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]

[ OK ] numpy version 1.21.1 is installed.
[ OK ] matplotlib version 3.4.2 is installed.
[ OK ] sklearn version 0.24.2 is installed.
[ OK ] pandas version 1.3.1 is installed.
[ OK ] xgboost version 1.3.3 is installed.
[ OK ] shap version 0.39.0 is installed.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pylab as plt
%pylab inline
from matplotlib import rcParams
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from  sklearn.linear_model import Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid

Populating the interactive namespace from numpy and matplotlib


pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy


In [3]:
fontsize=14
rcParams['font.size']=fontsize
rcParams['font.family']='sans-serif'
rcParams['axes.labelsize']=fontsize
rcParams['axes.titlesize']=fontsize
rcParams['xtick.labelsize']=12
rcParams['ytick.labelsize']=fontsize
rcParams['legend.fontsize']=fontsize
rcParams['figure.figsize']=(10,7.5)
rcParams['axes.grid']=True

In [4]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('data/train_V2.csv')
df.sample(10)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
df.shape

In [ ]:
match_df = df.groupby('matchId').count()['Id']

In [ ]:
fig = plt.figure()
plt.violinplot(match_df)
plt.ylabel('number of players')
plt.xlabel('match')
plt.title('Number of Players in Each Match')
plt.show()
fig.savefig('figures/Number of Players in Each Match.jpg', dpi = 300)

In [ ]:
fig,ax = plt.subplots(figsize=(16,12))
ax = sns.heatmap(df.corr(), annot=True)
fig.savefig('figures/Correlation.jpg', dpi = 300)

In [ ]:
fig = plt.figure()
df['winPlacePerc'].plot.hist(bins=100)
plt.xlabel('win place percentage')
plt.ylabel('count')
plt.title('Distribution of Win Place')
plt.show()
fig.savefig('figures/Distribution of Win Place.jpg', dpi = 300)

In [ ]:
fig = plt.figure()
df.plot.scatter('killPlace','winPlacePerc',s=1,alpha=0.1)
plt.ylabel('win place percentage')
plt.xlabel('kill place')
plt.title('Distribution of Win Place vs. Kill Place')
plt.show()
fig.savefig('figures/Distribution of Win Place vs. Kill Place.jpg', dpi = 300)

In [ ]:
fig = plt.figure()
df['walkDistance'].plot.hist(bins=50, log=True)
plt.xlabel('walk distance')
plt.ylabel('count')
plt.title('Distribution of Walk Distance')
plt.show()
fig.savefig('figures/Distribution of Walk Distance.jpg', dpi = 300)

In [ ]:
fig = plt.figure()
df['kills'].plot.hist(bins=df['kills'].nunique(), log=True)
plt.xlabel('kills')
plt.ylabel('count')
plt.title('Distribution of Kills')
plt.show()
fig.savefig('figures/Distribution of Kills.jpg', dpi = 300)

In [ ]:
fig = plt.figure()
df[['winPlacePerc','matchType']].boxplot(by='matchType',figsize=(22,16.5))
plt.ylabel('win place percentage')
plt.xlabel('match type')
plt.title('Distribution of Win Place')
plt.show()
fig.savefig('figures/Distribution of Win Place by Match Type.jpg', dpi = 300)

In [ ]:
fig = plt.figure()
pd.value_counts(df['matchType'],normalize=True).plot.bar()
plt.ylabel('fraction')
plt.xlabel('match type')
plt.title('Distribution of Match Type')
plt.show()
fig.savefig('figures/Distribution of Match Type.jpg', dpi = 300)

In [ ]:
fig = plt.figure()
pd.plotting.scatter_matrix(df[['assists', 'boosts', 'killPlace', 'kills', 'winPlacePerc']], marker='o', s=1, alpha=0.1)
plt.show()
fig.savefig('figures/Scatter Matrix.jpg', dpi = 300)

In [ ]:
n = len(pd.unique(df['Id']))
n

In [ ]:
column_list = ['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills', 
               'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace', 'revives', 'rideDistance', 
               'roadKills', 'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints', 
               'winPlacePerc']

In [ ]:
ndf = df[column_list]
ndf.sample(10)

In [ ]:
ndf.shape

In [ ]:
sdf = ndf.sample(round(0.01*len(ndf)), random_state=11)
sdf.to_csv('data/sample.csv')

In [5]:
sdf = pd.read_csv('data/sample.csv')
sdf.head()

,Unnamed: 0,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,2069048,0,0,57.33,0,0,0,82,0,0,0,0.00,1382,squad,27,0,0.0,0,0.0,0,0,199.2,2,0,0.1538
1,25967,3,5,217.10,1,0,2,22,1231,1,1,141.80,2091,squad,26,0,2405.0,0,0.0,0,1,3711.0,5,1530,0.9200
2,2896533,0,0,242.70,0,0,0,63,0,0,0,0.00,1361,squad-fpp,28,0,0.0,0,0.0,0,0,593.8,2,0,0.3704
3,3314281,0,0,100.00,1,0,0,31,0,1,1,13.73,1387,duo-fpp,48,0,0.0,0,11.3,0,0,1150.0,3,0,0.7234
4,2707511,0,0,63.21,0,0,0,81,0,0,0,0.00,1800,duo-fpp,49,0,0.0,0,0.0,0,0,94.5,2,0,0.1667


In [6]:
nsdf = sdf.loc[:, sdf.columns != 'Unnamed: 0']
nsdf.head()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,0,0,57.33,0,0,0,82,0,0,0,0.00,1382,squad,27,0,0.0,0,0.0,0,0,199.2,2,0,0.1538
1,3,5,217.10,1,0,2,22,1231,1,1,141.80,2091,squad,26,0,2405.0,0,0.0,0,1,3711.0,5,1530,0.9200
2,0,0,242.70,0,0,0,63,0,0,0,0.00,1361,squad-fpp,28,0,0.0,0,0.0,0,0,593.8,2,0,0.3704
3,0,0,100.00,1,0,0,31,0,1,1,13.73,1387,duo-fpp,48,0,0.0,0,11.3,0,0,1150.0,3,0,0.7234
4,0,0,63.21,0,0,0,81,0,0,0,0.00,1800,duo-fpp,49,0,0.0,0,0.0,0,0,94.5,2,0,0.1667


In [7]:
len(nsdf)

44470

In [8]:
X = nsdf.loc[:, nsdf.columns != 'winPlacePerc']
y = nsdf['winPlacePerc']

onehot_ftrs = ['matchType']
minmax_ftrs = ['killPlace', 'maxPlace']
std_ftrs = ['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'killPoints', 'kills', 'killStreaks', 
            'longestKill', 'matchDuration', 'revives', 'rideDistance', 'roadKills', 'swimDistance', 'teamKills', 
            'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), onehot_ftrs),
        ('minmax', MinMaxScaler(), minmax_ftrs),
        ('std', StandardScaler(), std_ftrs)])

prep = Pipeline(steps=[('preprocessor', preprocessor)])

In [9]:
alpha = np.logspace(-7,-1,7)
alpha

array([1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])

In [10]:
nr_states = 6
test_scores = np.zeros(nr_states)
final_models = []

for i in range(nr_states):
    X_train, X_other, y_train, y_other = train_test_split(X,y,train_size = 0.6,random_state=11*i)
    X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,random_state=11*i)
    
    X_train_prep = prep.fit_transform(X_train)
    X_val_prep = prep.transform(X_val)
    X_test_prep = prep.transform(X_test)
    
    train_score = np.zeros(len(alpha))
    val_score = np.zeros(len(alpha))
    models = []
    
    for p in range(len(alpha)):
        clf = Lasso(alpha=alpha[p],max_iter=100000000)
        clf.fit(X_train_prep, y_train)
        models.append(clf)
        
        y_train_pred = clf.predict(X_train_prep)
        y_val_pred = clf.predict(X_val_prep)
        y_train_pred = np.where(y_train_pred > 1.0, 1.0, y_train_pred)
        y_train_pred = np.where(y_train_pred < 0.0, 0.0, y_train_pred)
        y_val_pred = np.where(y_val_pred > 1.0, 1.0, y_val_pred)
        y_val_pred = np.where(y_val_pred < 0.0, 0.0, y_val_pred)
        
        train_score[p] = mean_absolute_error(y_train, y_train_pred)
        val_score[p] = mean_absolute_error(y_val, y_val_pred)
    
    print('best model parameters:',alpha[np.argmin(val_score)])
    print('corresponding validation score:',np.min(val_score))
    final_models.append(models[np.argmin(val_score)])
    y_test_pred = final_models[-1].predict(X_test_prep)
    y_test_pred = np.where(y_test_pred > 1.0, 1.0, y_test_pred)
    y_test_pred = np.where(y_test_pred < 0.0, 0.0, y_test_pred)
    test_scores[i] = mean_absolute_error(y_test, y_test_pred)
    print('test score:',test_scores[i])

best model parameters: 1e-07
corresponding validation score: 0.08951492183965114
test score: 0.08785433962237908
best model parameters: 1e-07
corresponding validation score: 0.08802668081343365
test score: 0.08797100619156659
best model parameters: 1e-07
corresponding validation score: 0.08743352569725793
test score: 0.08693195920063623
best model parameters: 1e-07
corresponding validation score: 0.08894966081738376
test score: 0.08916462432195785
best model parameters: 1e-07
corresponding validation score: 0.08863244137920218
test score: 0.08810369199894083
best model parameters: 1e-07
corresponding validation score: 0.08832557780293294
test score: 0.08785255577904047


In [11]:
lasso_models = final_models
lasso_models

[Lasso(alpha=1e-07, max_iter=100000000),
 Lasso(alpha=1e-07, max_iter=100000000),
 Lasso(alpha=1e-07, max_iter=100000000),
 Lasso(alpha=1e-07, max_iter=100000000),
 Lasso(alpha=1e-07, max_iter=100000000),
 Lasso(alpha=1e-07, max_iter=100000000)]

In [12]:
lasso_scores = test_scores
lasso_scores

array([0.08785434, 0.08797101, 0.08693196, 0.08916462, 0.08810369,
       0.08785256])

In [13]:
alpha = np.logspace(-7,-1,7)
alpha

array([1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])

In [14]:
nr_states = 6
test_scores = np.zeros(nr_states)
final_models = []

for i in range(nr_states):
    X_train, X_other, y_train, y_other = train_test_split(X,y,train_size = 0.6,random_state=11*i)
    X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,random_state=11*i)
    
    X_train_prep = prep.fit_transform(X_train)
    X_val_prep = prep.transform(X_val)
    X_test_prep = prep.transform(X_test)
    
    train_score = np.zeros(len(alpha))
    val_score = np.zeros(len(alpha))
    models = []
    
    for p in range(len(alpha)):
        clf = Ridge(alpha=alpha[p],max_iter=100000000)
        clf.fit(X_train_prep, y_train)
        models.append(clf)
        
        y_train_pred = clf.predict(X_train_prep)
        y_val_pred = clf.predict(X_val_prep)
        y_train_pred = np.where(y_train_pred > 1.0, 1.0, y_train_pred)
        y_train_pred = np.where(y_train_pred < 0.0, 0.0, y_train_pred)
        y_val_pred = np.where(y_val_pred > 1.0, 1.0, y_val_pred)
        y_val_pred = np.where(y_val_pred < 0.0, 0.0, y_val_pred)
        
        train_score[p] = mean_absolute_error(y_train, y_train_pred)
        val_score[p] = mean_absolute_error(y_val, y_val_pred)
    
    print('best model parameters:',alpha[np.argmin(val_score)])
    print('corresponding validation score:',np.min(val_score))
    final_models.append(models[np.argmin(val_score)])
    y_test_pred = final_models[-1].predict(X_test_prep)
    y_test_pred = np.where(y_test_pred > 1.0, 1.0, y_test_pred)
    y_test_pred = np.where(y_test_pred < 0.0, 0.0, y_test_pred)
    test_scores[i] = mean_absolute_error(y_test, y_test_pred)
    print('test score:',test_scores[i])

best model parameters: 1e-07
corresponding validation score: 0.08951472803879082
test score: 0.08785404549417367
best model parameters: 1e-07
corresponding validation score: 0.08802644156166324
test score: 0.08796424437316608
best model parameters: 1e-07
corresponding validation score: 0.0874332321190433
test score: 0.08692292643930062
best model parameters: 1e-07
corresponding validation score: 0.08894899843093694
test score: 0.08915692121863422
best model parameters: 1e-07
corresponding validation score: 0.08862746863184023
test score: 0.08810329436387354
best model parameters: 1e-07
corresponding validation score: 0.08832555865908474
test score: 0.0878522350920783


In [15]:
ridge_models = final_models
ridge_models

[Ridge(alpha=1e-07, max_iter=100000000),
 Ridge(alpha=1e-07, max_iter=100000000),
 Ridge(alpha=1e-07, max_iter=100000000),
 Ridge(alpha=1e-07, max_iter=100000000),
 Ridge(alpha=1e-07, max_iter=100000000),
 Ridge(alpha=1e-07, max_iter=100000000)]

In [16]:
ridge_scores = test_scores
ridge_scores

array([0.08785405, 0.08796424, 0.08692293, 0.08915692, 0.08810329,
       0.08785224])